In [60]:
import random 
import os
import math

from sage.all import *
import sys
import hashlib
from hashlib import sha3_256 as Hash1


In [61]:
class NTRU:
    def __init__(self):                                
        self.n = 677
        self.p = 3
        self.q = next_prime(self.p*self.n)

        self.iid_bits = 5408
        self.fixed_type_bits = 25688 
        
        #Criação dos anéis a utilizar
        Z.<w> = ZZ[]                                    
        phi4 = w - 1
        phi_n4 = (w^self.n - 1) / (w-1) 
        self.R = Z.quotient(phi4 * phi_n4)
        
        S.<x> = PolynomialRing(GF(3))
        phi_n = (x^self.n - 1) / (x - 1)
        self.S3 = QuotientRing(S, phi_n)

        SS.<y> = PolynomialRing(GF(self.q))
        phi_n2 = (y^self.n - 1) / (y-1)
        self.Sq = QuotientRing(SS, phi_n2)

        R.<z> = GF(self.q)[]
        phi3 = z - 1
        phi_n3 = (z^self.n - 1) / (z-1) 
        self.Rq = R.quotient(phi3 * phi_n3)
        
    #Função para arredondar os polinómios entre -1 e 1
    def Round(self,t, n=3):                           
        if n==-1:
            n=self.q
        Zx.<x>  = ZZ[]
        
        r = n//2
        res_list = []
        pol_list = t.list()
        for p in pol_list:
            res_list.append(lift(p+r) - r)
     
        return Zx(res_list)

    def Ternary(self, f_bits):
        v = 0
        i = 0
        while i < self.n-1:
            somatorio = 0 
            for j in range(7):
                somatorio += (2^j) * f_bits[8*i+j+1]
            v = v + somatorio * x^i
            i = i + 1
    
        v1 = self.S3(v)
        v = v1.lift().map_coefficients(lambda c: c.lift_centered(), ZZ)

        return v
    

    def FixedType(self, g_bits):
        A = [0]* (self.n-1)
        v = 0
        i = 0
        Zx.<x> = ZZ[]
        while i < (self.q//16) -1:
            somatorio = 0
            for j in range(29):
                somatorio += 2 ^(2+j) * g_bits[30*i+1+j]
            A[i] = 1 + somatorio
            i = i+1
            
        while i < (self.q//8)-2:
            somatorio = 0
            for j in range(29):
                somatorio += 2 ^(2+j) * g_bits[30*i+1+j]
            A[i] = 2 + somatorio
            i = i+1
            
        while i < self.n-1:
            somatorio = 0
            for j in range(29):
                somatorio += 2 ^(2+j) * g_bits[30*i+1+j]
            A[i] = 0 + somatorio
            i = i+ 1
        A.sort()
        i = 0
        while i < self.n-1:
            v = v + (A[i]%4)* x**i
            i +=1 
            
        v1 = self.S3(v)
        v = v1.lift().map_coefficients(lambda c: c.lift_centered(), ZZ)
    
        

        return v
            

        
    def sample_fg(self, seed):
        f_bits = seed[:self.iid_bits]
        g_bits = seed[self.iid_bits : self.iid_bits+ self.fixed_type_bits]
        f = self.Ternary(f_bits)
        g = self.FixedType(g_bits)
        return f, g
    
    
    
    def key_gen(self,seed):
        #(f , g) ← Sample_fg(seed)
        f, g = self.sample_fg(seed)   
        
        #2. fq ←(1/f)mod(q,Φn), ou seja, Calcular 1/f em S/q
        fq = self.Sq(f).inverse_of_unit()                   
        
        #3. h←(3·g·fq)mod(q,Φ1Φn), ou seja, Calcular 3 * g * f em R/q
        h = 3 * self.Rq(g) * self.Rq(fq.lift())              
         
        #4. hq ← (1/h) mod (q, Φn), ou seja, calcular hq em S/q
        hq = self.Sq(h.lift())^-1                            
        
        #5. fp ← (1/f) mod (3,Φn), ou seja, # Calcular fp em S3
        fp = self.S3(f).inverse_of_unit()                    
        
        #Chave priv = f, fp, hq
        privada = f, fp, hq
        #Chave publica = h
        publica = h
        return publica, privada
    

    def sample_rm(self,rm_bits): 
        #1. Parse rm_bits as r_bits ∥ m_bits with
        r_bits = seed[:self.iid_bits] 
        m_bits = seed[self.iid_bits: self.iid_bits + self.fixed_type_bits]

        #2. Set r = Ternary(r_bits)
        r = self.Ternary(r_bits)
        #3. Set m = Fixed_Type(m_bits)
        m = self.FixedType(m_bits)
        return r, m
    

    def encrypt(self, r, h, m):                             
        rh = self.Rq(r) * h                                  
        c = rh + self.Rq(m)
        b = self.Round(c,n=-1)
        return b
    
      # Verifica se um polinomio e ternario
    def isTernary(self, f):
        
        res = True
        v = list(f)
        for i in v:
            if i > 1 or i < -1:
                res = False
                break
        return res
    
    def decrypt(self, f, fp, hq, c):                       
        Zx.<x>  = ZZ[] 
        #a←(c·f)mod(q,Φ1Φn) = Calcular a em R/q
        a = self.Round(self.Rq(c) * self.Rq(f), n=-1)
        #m←(a·fp)mod(3,Φn) = Caclular m em S/3
        m = self.S3(a) * fp                                
        m_s3 = self.Round(m)
        
        
        # m′ ← Lift(m)
        #r←((c−m′)·hq)mod(q,Φn)  Calcular r em Sq
        r = (self.Sq(c) - self.Sq(m.lift())) * hq

        if not self.isTernary(r) and not self.isTernary(m_s3):
            return 0,0,1
        return r,m_s3,0

In [62]:
n = NTRU()
#Gerar a seed
seed = os.urandom(n.fixed_type_bits + n.iid_bits )

#Gerar a chave pública e privada
pub, priv = n.key_gen(seed)

#Gerar outra seed diferente
rm_bits = os.urandom(n.fixed_type_bits + n.iid_bits)

#Gerar polinómio r e m
r, m = n.sample_rm(rm_bits)

#Encrypt
c = n.encrypt(r,pub, m)
f, fp, hq = priv

#Decrypt
rr, pt, fail = n.decrypt(f, fp, hq, c)

#Verificação
if (m == pt):
    print("As mensagens coincidem!")
else : 
    print("Processo de cifragem e decifragem falhou!")

As mensagens coincidem!


In [69]:
class NTRU(NTRU):
    
    def hash1(self, M):                                  
        m = Hash1()
        m.update(M)
        return m.digest()
    
    
    def hash2(self, M):                                  
        m = Hash1()
        m.update(M)
        return m.digest()
    
    def toZZ(self,f,p=None):                                   #Função que arredonda
        ff = list(f)
        if p == None:
            return ff
        else:
            fp = map(lift,[Mod(a,p) for a in ff])
        return [u if u <= p//2 else u-p for u in fp ]
    
    def key_gen_kem(self, seed):    
        #1. (f, fp, hq), h) ← KeyGen′(seed)
        pub, priv = self.key_gen(seed)
        #2. s ←$ {0, 1}256
        s = os.urandom(256)
        #8. return((f,fp,hq,s),h)
        f, fp, hq = priv 
        priv1 = f, fp, hq , s
        return pub, priv1
    
    def encaps(self, h):             
        #1. coins ←$ {0, 1}256
        coins = os.urandom(256)
        #2. (r, m) ← Sample_rm(coins)
        r, m = self.sample_rm(coins)
        #3. c ← Encrypt(h, (r, m))
        c = self.encrypt(r, h, m)
        
        #4. k←H1(r,m)
        r_bytes = str(self.toZZ(c)).encode('utf-8')
        m_bytes = str(self.toZZ(m)).encode('utf-8')
        rm_packed = r_bytes + m_bytes
        
        k = self.hash1(rm_packed)
        #5. return (c, k)
        return c, k
    
    def decaps(self, priv, c):    
        #1. (r,m,fail) ← Decrypt((f,fp,hq),c)
        f, fp, hq, s = priv
        r, m, fail = self.decrypt(f, fp, hq, c)
        
        #2.k1←H1(r,m)
        r_bytes = str(self.toZZ(c)).encode('utf-8')
        m_bytes = str(self.toZZ(m)).encode('utf-8')
        rm_packed = r_bytes + m_bytes
        k1 = self.hash1(rm_packed)
        
        #3.k2←H2(s,c)
        c_bytes = str(self.toZZ(c)).encode('utf-8')
        k2 = self.hash1(s + c_bytes)
        
        #4.if fail=0 return k1
        if fail == 0:
            return k1 
        #5. else return k2
        else:
            return k2
        
    

In [70]:
ntru = NTRU()
#Gerar a seed
seed = os.urandom(ntru.fixed_type_bits + ntru.iid_bits )
#Gerar a chave pública e privada
pub, priv = ntru.key_gen_kem(seed)
#Encapsular
c, k = ntru.encaps(pub)
#Desencapsular
kk = ntru.decaps(priv, c)
#Verificação
print(k == kk)



True
